![logo](./finspace_logo.png)

In [ ]:
%local
from aws.finspace.cluster import FinSpaceClusterManager

# if this was already run, no need to run again
if 'finspace_clusters' not in globals():
    finspace_clusters = FinSpaceClusterManager()
    finspace_clusters.auto_connect()
else:
    print(f'connected to cluster: {finspace_clusters.get_connected_cluster_id()}')

## Configure Spark for Snowflake
This ensures the cluster gets the maven packages deployed to it so the cluster can communicate with Snowflake. The '-f' argument below will force any running spark session to restart.

In [ ]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "net.snowflake:snowflake-jdbc:3.13.5,net.snowflake:spark-snowflake_2.11:2.9.0-spark_2.4"
    }
}

In [ ]:
%local
import configparser

# read the config
config = configparser.ConfigParser()
config.read("snowflake.ini")

# values from config
snowflake_user=config['snowflake']['user']
snowflake_password=config['snowflake']['password']
snowflake_account=config['snowflake']['account']
snowflake_database=config['snowflake']['database']
snowflake_warehouse=config['snowflake']['warehouse']

print(f"""snowflake_user={snowflake_user}
snowflake_password={snowflake_password}
snowflake_account={snowflake_account}
snowflake_database={snowflake_database}
snowflake_warehouse={snowflake_warehouse}
""")

In [ ]:
%send_to_spark -i snowflake_user

In [ ]:
%send_to_spark -i snowflake_password 

In [ ]:
%send_to_spark -i snowflake_account

In [ ]:
%send_to_spark -i snowflake_database

In [ ]:
%send_to_spark -i snowflake_warehouse

In [ ]:
# Snowflake options for the spark data source
# username and password should be protected, admitedly in the clear for convenience
sfOptions = {
    "sfURL" : f"{snowflake_account}.snowflakecomputing.com",
    "sfUser" : snowflake_user,
    "sfPassword" : snowflake_password,
    "sfDatabase" : snowflake_database,
    "sfWarehouse" : snowflake_warehouse,
    "autopushdown": "on",
    "keep_column_case": "on"
}

# class name for the snowflake spark data source
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

# Python Helper Classes
These are the FinSpace helper classes found in the samples and examples github

In [2]:
# %load finspace.py
import datetime
import time
import boto3
import os
import pandas as pd
import urllib

from urllib.parse import urlparse
from botocore.config import Config
from boto3.session import Session


# Base FinSpace class
class FinSpace:

    def __init__(
            self,
            config=Config(retries={'max_attempts': 3, 'mode': 'standard'}),
            boto_session: Session = None,
            dev_overrides: dict = None,
            service_name = 'finspace-data'):
        """
        To configure this class object, simply instantiate with no-arg if hitting prod endpoint, or else override it:
        e.g.
           `hab = FinSpaceAnalyticsManager(region_name = 'us-east-1',
           dev_overrides = {'hfs_endpoint': 'https://39g32x40jk.execute-api.us-east-1.amazonaws.com/alpha'})`
        """
        self.hfs_endpoint = None
        self.region_name = None

        if dev_overrides is not None:
            if 'hfs_endpoint' in dev_overrides:
                self.hfs_endpoint = dev_overrides['hfs_endpoint']

            if 'region_name' in dev_overrides:
                self.region_name = dev_overrides['region_name']
        else:
            if boto_session is not None:
                self.region_name = boto_session.region_name
            else:
                self.region_name = self.get_region_name()

        self.config = config

        self._boto3_session = boto3.session.Session(region_name=self.region_name) if boto_session is None else boto_session

        print(f"service_name: {service_name}")
        print(f"endpoint: {self.hfs_endpoint}")
        print(f"region_name: {self.region_name}")

        self.client = self._boto3_session.client(service_name, endpoint_url=self.hfs_endpoint, config=self.config)

    @staticmethod
    def get_region_name():
        req = urllib.request.Request("http://169.254.169.254/latest/meta-data/placement/region")
        with urllib.request.urlopen(req) as response:
            return response.read().decode("utf-8")

    # --------------------------------------
    # Utility Functions
    # --------------------------------------
    @staticmethod
    def get_list(all_list: dir, name: str):
        """
        Search for name found in the all_list dir and return that list of things.
        Removes repetitive code found in functions that call boto apis then search for the expected returned items

        :param all_list: list of things to search
        :type: dir:

        :param name: name to search for in all_lists
        :type: str

        :return: list of items found in name
        """
        r = []

        # is the given name found, is found, add to list
        if name in all_list:
            for s in all_list[name]:
                r.append(s)

        # return the list
        return r

    # --------------------------------------
    # Classification Functions
    # --------------------------------------

    def list_classifications(self):
        """
        Return list of all classifications

        :return: all classifications
        """
        all_list = self.client.list_classifications(sort='NAME')

        return self.get_list(all_list, 'classifications')

    def classification_names(self):
        """
        Get the classifications names

        :return list of classifications names only
        """
        classification_names = []
        all_classifications = self.list_classifications()
        for c in all_classifications:
            classification_names.append(c['name'])
        return classification_names

    def classification(self, name: str):
        """
        Exact name search for a classification of the given name

        :param name: name of the classification to find
        :type: str

        :return
        """

        all_classifications = self.list_classifications()
        existing_classification = next((c for c in all_classifications if c['name'].lower() == name.lower()), None)
        if existing_classification:
            return existing_classification

    def describe_classification(self, classification_id: str):
        """
        Calls the describe classification API function and only returns the taxonomy portion of the response.

        :param classification_id: the GUID of the classification to get description of
        :type: str
        """
        resp = None
        taxonomy_details_resp = self.client.describe_taxonomy(taxonomyId=classification_id)

        if 'taxonomy' in taxonomy_details_resp:
            resp = taxonomy_details_resp['taxonomy']

        return (resp)

    def create_classification(self, classification_definition):
        resp = self.client.create_taxonomy(taxonomyDefinition=classification_definition)

        taxonomy_id = resp["taxonomyId"]

        return (taxonomy_id)

    def delete_classification(self, classification_id):
        resp = self.client.delete_taxonomy(taxonomyId=classification_id)

        if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
            return resp

        return True

    # --------------------------------------
    # Attribute Set Functions
    # --------------------------------------

    def list_attribute_sets(self):
        """
        Get list of all dataset_types in the system

        :return: list of dataset types
        """
        resp = self.client.list_dataset_types()
        results = resp['datasetTypeSummaries']

        while "nextToken" in resp:
            resp = self.client.list_dataset_types(nextToken=resp['nextToken'])
            results.extend(resp['datasetTypeSummaries'])

        return (results)

    def attribute_set_names(self):
        """
        Get the list of all dataset type names

        :return list of all dataset type names
        """

        dataset_type_names = []
        all_dataset_types = self.list_dataset_types()
        for c in all_dataset_types:
            dataset_type_names.append(c['name'])
        return dataset_type_names

    def attribute_set(self, name: str):
        """
        Exact name search for a dataset type of the given name

        :param name: name of the dataset type to find
        :type: str

        :return
        """

        all_dataset_types = self.list_dataset_types()
        existing_dataset_type = next((c for c in all_dataset_types if c['name'].lower() == name.lower()), None)
        if existing_dataset_type:
            return existing_dataset_type

    def describe_attribute_set(self, attribute_set_id: str):
        """
        Calls the describe dataset type API function and only returns the dataset type portion of the response.

        :param attribute_set_id: the GUID of the dataset type to get description of
        :type: str
        """
        resp = None
        dataset_type_details_resp = self.client.describe_dataset_type(datasetTypeId=attribute_set_id)

        if 'datasetType' in dataset_type_details_resp:
            resp = dataset_type_details_resp['datasetType']

        return (resp)

    def create_attribute_set(self, attribute_set_def):
        resp = self.client.create_dataset_type(datasetTypeDefinition=attribute_set_def)

        att_id = resp["datasetTypeId"]

        return (att_id)

    def delete_attribute_set(self, attribute_set_id: str):
        resp = self.client.delete_attribute_set(attributeSetId=attribute_set_id)

        if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
            return resp

        return True

    def associate_attribute_set(self, att_name: str, att_values: list, dataset_id: str):
        fncs = ['dissociate_dataset_from_attribute_set', 'associate_dataset_with_attribute_set', 'update_dataset_attribute_set_context']
        if self.check_functions(fncs) is False:
            raise Exception(f"not all functions found in client {fncs}")

        # get the attribute set by name, will need its id
        att_set = self.attribute_set(att_name)

        # get the dataset's information, will need the arn
        dataset = self.describe_dataset_details(dataset_id=dataset_id)

        # disassociate any existing relationship
        try:
            self.client.dissociate_dataset_from_attribute_set(datasetArn=dataset['arn'],
                                                             datasetTypeId=att_set['id'])
        except:
            print("Nothing to disassociate")

        arn = dataset['arn']
        dataset_type_id = att_set['id']

        self.client.associate_dataset_with_attribute_set(datasetId=dataset_id, datasetArn=arn, attributeSetId=dataset_type_id)

        resp = self.client.update_dataset_attribute_set_context(datasetId=dataset_id, datasetArn=arn, attributeSetId=dataset_type_id, values=att_values)

        if resp['ResponseMetadata']['HTTPStatusCode'] != 200:
            return resp

        return True
    
    # --------------------------------------
    # Permission Group Functions
    # --------------------------------------

    def list_permission_groups(self, max_results: int):
        all_perms = self.client.list_permission_groups(MaxResults=max_results)
        return (self.get_list(all_perms, 'permissionGroups'))

    def permission_group(self, name):
        all_groups = self.list_permission_groups(max_results = 100)

        existing_group = next((c for c in all_groups if c['name'].lower() == name.lower()), None)

        if existing_group:
            return existing_group

    def describe_permission_group(self, permission_group_id: str):
        resp = None

        perm_resp = self.client.describe_permission_group(permissionGroupId=permission_group_id)

        if 'permissionGroup' in perm_resp:
            resp = perm_resp['permissionGroup']

        return (resp)

    # --------------------------------------
    # Dataset Functions
    # --------------------------------------

    def describe_dataset_details(self, dataset_id: str):
        """
        Calls the describe dataset details API function and only returns the dataset details portion of the response.

        :param dataset_id: the GUID of the dataset to get description of
        :type: str
        """
        resp = None
        dataset_details_resp = self.client.describe_dataset_details(datasetId=dataset_id)

        if 'dataset' in dataset_details_resp:
            resp = dataset_details_resp["dataset"]

        return (resp)

    def create_dataset(self, name: str, description: str, permission_group_id: str, dataset_permissions: [], kind: str,
                       owner_info, schema):
        """
        Create a dataset

        Warning, dataset names are not unique, be sure to check for the same name dataset before creating a new one

        :param name: Name of the dataset
        :type: str

        :param description: Description of the dataset
        :type: str

        :param permission_group_id: permission group for the dataset
        :type: str

        :param dataset_permissions: permissions for the group on the dataset

        :param kind: Kind of dataset, choices: TABULAR
        :type: str

        :param owner_info: owner information for the dataset

        :param schema: Schema of the dataset

        :return: the dataset_id of the created dataset
        """

        if dataset_permissions:
            request_dataset_permissions = [{"permission": permissionName} for permissionName in dataset_permissions]
        else:
            request_dataset_permissions = []

        response = self.client.create_dataset(name=name,
                                              permissionGroupId = permission_group_id,
                                              datasetPermissions = request_dataset_permissions,
                                              kind=kind,
                                              description = description.replace('\n', ' '),
                                              ownerInfo = owner_info,
                                              schema = schema)

        return response["datasetId"]

    def ingest_from_s3(self,
                       s3_location: str,
                       dataset_id: str,
                       change_type: str,
                       wait_for_completion: bool = True,
                       format_type: str = "CSV",
                       format_params: dict = {'separator': ',', 'withHeader': 'true'}):
        """
        Creates a changeset and ingests the data given in the S3 location into the changeset

        :param s3_location: the source location of the data for the changeset, will be copied into the changeset
        :stype: str

        :param dataset_id: the identifier of the containing dataset for the changeset to be created for this data
        :type: str

        :param change_type: What is the kind of changetype?  "APPEND", "REPLACE" are the choices
        :type: str

        :param wait_for_completion: Boolean, should the function wait for the operation to complete?
        :type: str

        :param format_type: format type, CSV, PARQUET, XML, JSON
        :type: str

        :param format_params: dictionary of format parameters
        :type: dict

        :return: the id of the changeset created
        """
        create_changeset_response = self.client.create_changeset(
            datasetId=dataset_id,
            changeType=change_type,
            sourceType='S3',
            sourceParams={'s3SourcePath': s3_location},
            formatType=format_type.upper(),
            formatParams=format_params
        )

        changeset_id = create_changeset_response['changeset']['id']

        if wait_for_completion:
            self.wait_for_ingestion(dataset_id, changeset_id)
        return changeset_id

    def describe_changeset(self, dataset_id: str, changeset_id: str):
        """
        Function to get a description of the the givn changeset for the given dataset

        :param dataset_id: identifier of the dataset
        :type: str

        :param changeset_id: the idenfitier of the changeset
        :type: str

        :return: all information about the changeset, if found
        """
        describe_changeset_resp = self.client.describe_changeset(datasetId=dataset_id, id=changeset_id)

        return describe_changeset_resp['changeset']

    def create_as_of_view(self, dataset_id: str, as_of_date: datetime, destination_type: str,
                          partition_columns: list = [], sort_columns: list = [], destination_properties: dict = {},
                          wait_for_completion: bool = True):
        """
        Creates an 'as of' static view up to and including the requested 'as of' date provided.

        :param dataset_id: identifier of the dataset
        :type: str

        :param as_of_date: as of date, will include changesets up to this date/time in the view
        :type: datetime

        :param destination_type: destination type
        :type: str

        :param partition_columns: columns to partition the data by for the created view
        :type: list

        :param sort_columns: column to sort the view by
        :type: list

        :param destination_properties: destination properties
        :type: dict

        :param wait_for_completion: should the function wait for the system to create the view?
        :type: bool

        :return str: GUID of the created view if successful

        """
        create_materialized_view_resp = self.client.create_materialized_snapshot(
            datasetId=dataset_id,
            asOfTimestamp=as_of_date,
            destinationType=destination_type,
            partitionColumns=partition_columns,
            sortColumns=sort_columns,
            autoUpdate=False,
            destinationProperties=destination_properties
        )
        view_id = create_materialized_view_resp['id']
        if wait_for_completion:
            self.wait_for_view(dataset_id=dataset_id, view_id=view_id)
        return view_id

    def create_auto_update_view(self, dataset_id: str, destination_type: str,
                                partition_columns=[], sort_columns=[], destination_properties={},
                                wait_for_completion=True):
        """
        Creates an auto-updating view of the given dataset

        :param dataset_id: identifier of the dataset
        :type: str

        :param destination_type: destination type
        :type: str

        :param partition_columns: columns to partition the data by for the created view
        :type: list

        :param sort_columns: column to sort the view by
        :type: list

        :param destination_properties: destination properties
        :type: str

        :param wait_for_completion: should the function wait for the system to create the view?
        :type: bool

        :return str: GUID of the created view if successful

        """
        create_materialized_view_resp = self.client.create_materialized_snapshot(
            datasetId=dataset_id,
            destinationType=destination_type,
            partitionColumns=partition_columns,
            sortColumns=sort_columns,
            autoUpdate=True,
            destinationProperties=destination_properties
        )
        view_id = create_materialized_view_resp['id']
        if wait_for_completion:
            self.wait_for_view(dataset_id=dataset_id, view_id=view_id)
        return view_id

    def wait_for_ingestion(self, dataset_id: str, changeset_id: str, sleep_sec=10):
        """
        function that will continuously poll the changeset creation to ensure it completes or fails before returning.

        :param dataset_id: GUID of the dataset
        :type: str

        :param changeset_id: GUID of the changeset
        :type: str

        :param sleep_sec: seconds to wait between checks
        :type: int

        """
        while True:
            status = self.describe_changeset(dataset_id=dataset_id, changeset_id=changeset_id)['status']
            if status == 'SUCCESS':
                print(f"Changeset complete")
                break
            elif status == 'PENDING' or status == 'RUNNING':
                print(f"Changeset status is still PENDING, waiting {sleep_sec} sec ...")
                time.sleep(sleep_sec)
                continue
            else:
                raise Exception(f"Bad changeset status: {status}, failing now.")

    def wait_for_view(self, dataset_id: str, view_id: str, sleep_sec=10):
        """
        function that will continuously poll the view creation to ensure it completes or fails before returning.

        :param dataset_id: GUID of the dataset
        :type: str

        :param view_id: GUID of the view
        :type: str

        :param sleep_sec: seconds to wait between checks
        :type: int

        """
        while True:
            list_views_resp = self.client.list_materialization_snapshots(datasetId=dataset_id, maxResults=100)
            matched_views = list(filter(lambda d: d['id'] == view_id, list_views_resp['materializationSnapshots']))

            if len(matched_views) != 1:
                size = len(matched_views)
                raise Exception(f"Unexpected error: found {size} views that match the view Id: {view_id}")

            status = matched_views[0]['status']
            if status == 'SUCCESS':
                print(f"View complete")
                break
            elif status == 'PENDING' or status == 'RUNNING':
                print(f"View status is still PENDING, continue to wait till finish...")
                time.sleep(sleep_sec)
                continue
            else:
                raise Exception(f"Bad view status: {status}, failing now.")

    def list_changesets(self, dataset_id: str):
        resp = self.client.list_changesets(datasetId=dataset_id, sortKey='CREATE_TIMESTAMP')
        results = resp['changesets']

        while "nextToken" in resp:
            resp = self.client.list_changesets(datasetId=dataset_id, sortKey='CREATE_TIMESTAMP',
                                               nextToken=resp['nextToken'])
            results.extend(resp['changesets'])

        return (results)

    def list_views(self, dataset_id: str, max_results=50):
        resp = self.client.list_materialization_snapshots(datasetId=dataset_id, maxResults=max_results)
        results = resp['materializationSnapshots']

        while "nextToken" in resp:
            resp = self.client.list_materialization_snapshots(datasetId=dataset_id, maxResults=max_results,
                                                              nextToken=resp['nextToken'])
            results.extend(resp['materializationSnapshots'])

        return (results)

    def list_datasets(self, max_results: int):
        all_datasets = self.client.list_datasets(maxResults=max_results)
        return (self.get_list(all_datasets, 'datasets'))

    def list_dataset_types(self):
        resp = self.client.list_dataset_types(sort='NAME')
        results = resp['datasetTypeSummaries']

        while "nextToken" in resp:
            resp = self.client.list_dataset_types(sort='NAME', nextToken=resp['nextToken'])
            results.extend(resp['datasetTypeSummaries'])

        return (results)

    @staticmethod
    def get_execution_role():
        """
        Convenience function from SageMaker to get the execution role of the user of the sagemaker studio notebook

        :return: the ARN of the execution role in the sagemaker studio notebook
        """
        import sagemaker as sm

        e_role = sm.get_execution_role()
        return (f"{e_role}")

    def get_user_ingestion_info(self):
        return (self.client.get_user_ingestion_info())

    def upload_pandas(self, data_frame: pd.DataFrame):
        import awswrangler as wr
        resp = self.client.get_working_location(locationType='INGESTION')
        upload_location = resp['s3Uri']
        wr.s3.to_parquet(data_frame, f"{upload_location}data.parquet", index=False, boto3_session=self._boto3_session)
        return upload_location

    def ingest_pandas(self, data_frame: pd.DataFrame, dataset_id: str, change_type: str, wait_for_completion=True):
        print("Uploading the pandas dataframe ...")
        upload_location = self.upload_pandas(data_frame)

        print("Data upload finished. Ingesting data ...")
        return self.ingest_from_s3(upload_location, dataset_id, change_type, wait_for_completion, format_type='PARQUET')

    def read_view_as_pandas(self, dataset_id: str, view_id: str):
        """
        Returns a pandas dataframe the view of the given dataset.  Views in FinSpace can be quite large, be careful!

        :param dataset_id:
        :param view_id:

        :return: Pandas dataframe with all data of the view
        """
        import awswrangler as wr  # use awswrangler to read the table

        # @todo: switch to DescribeMateriliazation when available in HFS
        views = self.list_views(dataset_id=dataset_id, max_results=50)
        filtered = [v for v in views if v['id'] == view_id]

        if len(filtered) == 0:
            raise Exception('No such view found')
        if len(filtered) > 1:
            raise Exception('Internal Server error')
        view = filtered[0]

        # 0. Ensure view is ready to be read
        if (view['status'] != 'SUCCESS'):
            status = view['status']
            print(f'view run status is not ready: {status}. Returning empty.')
            return

        glue_db_name = view['destinationTypeProperties']['databaseName']
        glue_table_name = view['destinationTypeProperties']['tableName']

        # determine if the table has partitions first, different way to read is there are partitions
        p = wr.catalog.get_partitions(table=glue_table_name, database=glue_db_name, boto3_session=self._boto3_session)

        def no_filter(partitions):
            if len(partitions.keys()) > 0:
                return True

            return False

        df = None

        if len(p) == 0:
            df = wr.s3.read_parquet_table(table=glue_table_name, database=glue_db_name,
                                          boto3_session=self._boto3_session)
        else:
            spath = wr.catalog.get_table_location(table=glue_table_name, database=glue_db_name,
                                                  boto3_session=self._boto3_session)
            cpath = wr.s3.list_directories(f"{spath}/*", boto3_session=self._boto3_session)

            read_path = f"{spath}/"

            # just one?  Read it
            if len(cpath) == 1:
                read_path = cpath[0]

            df = wr.s3.read_parquet(read_path, dataset=True, partition_filter=no_filter,
                                    boto3_session=self._boto3_session)

        # Query Glue table directly with wrangler
        return df

    @staticmethod
    def get_schema_from_pandas(df: pd.DataFrame):
        """
        Returns the FinSpace schema columns from the given pandas dataframe.

        :param df: pandas dataframe to interrogate for the schema

        :return: FinSpace column schema list
        """

        # for translation to FinSpace's schema
        # 'STRING'|'CHAR'|'INTEGER'|'TINYINT'|'SMALLINT'|'BIGINT'|'FLOAT'|'DOUBLE'|'DATE'|'DATETIME'|'BOOLEAN'|'BINARY'
        DoubleType = "DOUBLE"
        FloatType = "FLOAT"
        DateType = "DATE"
        StringType = "STRING"
        IntegerType = "INTEGER"
        LongType = "BIGINT"
        BooleanType = "BOOLEAN"
        TimestampType = "DATETIME"

        hab_columns = []

        for name in dict(df.dtypes):
            p_type = df.dtypes[name]

            switcher = {
                "float64": DoubleType,
                "int64": IntegerType,
                "datetime64[ns, UTC]": TimestampType,
                "datetime64[ns]": DateType
            }

            habType = switcher.get(str(p_type), StringType)

            hab_columns.append({
                "dataType": habType,
                "name": name,
                "description": ""
            })

        return (hab_columns)

    @staticmethod
    def get_date_cols(df: pd.DataFrame):
        """
        Returns which are the data columns found in the pandas dataframe.
        Pandas does the hard work to figure out which of the columns can be considered to be date columns.

        :param df: pandas dataframe to interrogate for the schema

        :return: list of column names that can be parsed as dates by pandas

        """
        date_cols = []

        for name in dict(df.dtypes):

            p_type = df.dtypes[name]

            if str(p_type).startswith("date"):
                date_cols.append(name)

        return (date_cols)

    def get_best_schema_from_csv(self, path, is_s3=True, read_rows=500, sep=','):
        """
        Uses multiple reads of the file with pandas to determine schema of the referenced files.
        Files are expected to be csv.

        :param path: path to the files to read
        :type: str

        :param is_s3: True if the path is s3;  False if filesystem
        :type: bool

        :param read_rows: number of rows to sample for determining schema

        :param sep:

        :return dict: schema for FinSpace
        """
        #
        # best efforts to determine the schema, sight unseen
        import awswrangler as wr

        # 1: get the base schema
        df1 = None

        if is_s3:
            df1 = wr.s3.read_csv(path, nrows=read_rows, sep=sep)
        else:
            df1 = pd.read_csv(path, nrows=read_rows, sep=sep)

        num_cols = len(df1.columns)

        # with number of columns, try to infer dates
        df2 = None

        if is_s3:
            df2 = wr.s3.read_csv(path, parse_dates=list(range(0, num_cols)), infer_datetime_format=True,
                                 nrows=read_rows, sep=sep)
        else:
            df2 = pd.read_csv(path, parse_dates=list(range(0, num_cols)), infer_datetime_format=True, nrows=read_rows,
                              sep=sep)

        date_cols = self.get_date_cols(df2)

        # with dates known, parse the file fully
        df = None

        if is_s3:
            df = wr.s3.read_csv(path, parse_dates=date_cols, infer_datetime_format=True, nrows=read_rows, sep=sep)
        else:
            df = pd.read_csv(path, parse_dates=date_cols, infer_datetime_format=True, nrows=read_rows, sep=sep)

        schema_cols = self.get_schema_from_pandas(df)

        return (schema_cols)

    def s3_upload_file(self, source_file: str, s3_destination: str):
        """
        Uploads a local file (full path) to the s3 destination given (expected form: s3://<bucket>/<prefix>/).
        The filename will have spaces replaced with _.

        :param source_file: path of file to upload
        :param s3_destination: full path to where to save the file
        :type: str

        """
        hab_s3_client = self._boto3_session.client(service_name='s3')

        o = urlparse(s3_destination)
        bucket = o.netloc
        prefix = o.path.lstrip('/')

        fname = os.path.basename(source_file)

        hab_s3_client.upload_file(source_file, bucket, f"{prefix}{fname.replace(' ', '_')}")

    def list_objects(self, s3_location: str):
        """
        lists the objects found at the s3_location. Strips out the boto API response header,
        just returns the contents of the location. Internally uses the list_objects_v2.

        :param s3_location: path, starting with s3:// to get the list of objects from
        :type: str

        """
        o = urlparse(s3_location)
        bucket = o.netloc
        prefix = o.path.lstrip('/')

        results = []

        hab_s3_client = self._boto3_session.client(service_name='s3')

        paginator = hab_s3_client.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket, Prefix=prefix)

        for page in pages:
            if 'Contents' in page:
                results.extend(page['Contents'])

        return (results)

    def list_clusters(self, status: str = None):
        """
        Lists current clusters and their statuses

        :param status: status to filter for

        :return dict: list of clusters
        """

        resp = self.client.list_clusters()

        clusters = []

        if 'clusters' not in resp:
            return (clusters)

        for c in resp['clusters']:
            if status is None:
                clusters.append(c)
            else:
                if c['clusterStatus']['state'] in status:
                    clusters.append(c)

        return (clusters)

    def get_cluster(self, cluster_id):
        """
        Resize the given cluster to desired template

        :param cluster_id: cluster id
        """

        clusters = self.list_clusters()

        for c in clusters:
            if c['clusterId'] == cluster_id:
                return (c)

        return (None)

    def update_cluster(self, cluster_id: str, template: str):
        """
        Resize the given cluster to desired template

        :param cluster_id: cluster id
        :param template: target template to resize to
        """

        cluster = self.get_cluster(cluster_id=cluster_id)

        if cluster['currentTemplate'] == template:
            print(f"Already using template: {template}")
            return (cluster)

        self.client.update_cluster(clusterId=cluster_id, template=template)

        return (self.get_cluster(cluster_id=cluster_id))

    def wait_for_status(self, clusterId: str, status: str, sleep_sec=10, max_wait_sec=900):
        """
        Function polls service until cluster is in desired status.

        :param clusterId: the cluster's ID
        :param status: desired status for clsuter to reach
        :
        """
        total_wait = 0

        while True and total_wait < max_wait_sec:
            resp = self.client.list_clusters()

            this_cluster = None

            # is this the cluster?
            for c in resp['clusters']:
                if clusterId == c['clusterId']:
                    this_cluster = c

            if this_cluster is None:
                print(f"clusterId:{clusterId} not found")
                return (None)

            this_status = this_cluster['clusterStatus']['state']

            if this_status.upper() != status.upper():
                print(f"Cluster status is {this_status}, waiting {sleep_sec} sec ...")
                time.sleep(sleep_sec)
                total_wait = total_wait + sleep_sec
                continue
            else:
                return (this_cluster)

    def get_working_location(self, locationType='SAGEMAKER'):
        resp = None
        location = self.client.get_working_location(locationType=locationType)

        if 's3Uri' in location:
            resp = location['s3Uri']

        return (resp)


In [ ]:
# %load finspace_spark.py
import datetime
import time
import boto3
from botocore.config import Config

# FinSpace class with Spark bindings

class SparkFinSpace(FinSpace):
    import pyspark
    def __init__(
        self, 
        spark: pyspark.sql.session.SparkSession = None,
        config = Config(retries = {'max_attempts': 0, 'mode': 'standard'}),
        dev_overrides: dict = None
    ):
        FinSpace.__init__(self, config=config, dev_overrides=dev_overrides)
        self.spark = spark # used on Spark cluster for reading views, creating changesets from DataFrames
        
    def upload_dataframe(self, data_frame: pyspark.sql.dataframe.DataFrame):
        resp = self.client.get_user_ingestion_info()
        upload_location = resp['ingestionPath']
#        data_frame.write.option('header', 'true').csv(upload_location)
        data_frame.write.parquet(upload_location)
        return upload_location
    
    def ingest_dataframe(self, data_frame: pyspark.sql.dataframe.DataFrame, dataset_id: str, change_type: str, wait_for_completion=True):
        print("Uploading data...")
        upload_location = self.upload_dataframe(data_frame)
        
        print("Data upload finished. Ingesting data...")
        
        return self.ingest_from_s3(upload_location, dataset_id, change_type, wait_for_completion, format_type='parquet', format_params={})
    
    def read_view_as_spark(
        self,
        dataset_id: str,
        view_id: str
        ):
        # TODO: switch to DescribeMatz when available in HFS
        views = self.list_views(dataset_id=dataset_id, max_results=50)
        filtered = [v for v in views if v['id'] == view_id]

        if len(filtered) == 0:
            raise Exception('No such view found')
        if len(filtered) > 1:
            raise Exception('Internal Server error')
        view = filtered[0]
        
        # 0. Ensure view is ready to be read
        if (view['status'] != 'SUCCESS'): 
            status = view['status'] 
            print(f'view run status is not ready: {status}. Returning empty.')
            return

        glue_db_name = view['destinationTypeProperties']['databaseName']
        glue_table_name = view['destinationTypeProperties']['tableName']
        
        # Query Glue table directly with catalog function of spark
        return self.spark.table(f"`{glue_db_name}`.`{glue_table_name}`")
    
    def get_schema_from_spark(self, data_frame: pyspark.sql.dataframe.DataFrame):
        from pyspark.sql.types import StructType

        # for translation to FinSpace's schema
        # 'STRING'|'CHAR'|'INTEGER'|'TINYINT'|'SMALLINT'|'BIGINT'|'FLOAT'|'DOUBLE'|'DATE'|'DATETIME'|'BOOLEAN'|'BINARY'
        DoubleType    = "DOUBLE"
        FloatType     = "FLOAT"
        DateType      = "DATE"
        StringType    = "STRING"
        IntegerType   = "INTEGER"
        LongType      = "BIGINT"
        BooleanType   = "BOOLEAN"
        TimestampType = "DATETIME"
        
        hab_columns = []

        items = [i for i in data_frame.schema] 

        switcher = {
            "BinaryType"    : StringType,
            "BooleanType"   : BooleanType,
            "ByteType"      : IntegerType,
            "DateType"      : DateType,
            "DoubleType"    : FloatType,
            "IntegerType"   : IntegerType,
            "LongType"      : IntegerType,
            "NullType"      : StringType,
            "ShortType"     : IntegerType,
            "StringType"    : StringType,
            "TimestampType" : TimestampType,
        }

        
        for i in items:
#            print( f"name: {i.name} type: {i.dataType}" )

            habType = switcher.get( str(i.dataType), StringType)

            hab_columns.append({
                "dataType"    : habType, 
                "name"        : i.name,
                "description" : ""
            })

        return( hab_columns )


In [ ]:
# initialize the FinSpace helper
finspace = SparkFinSpace(spark=spark)

# Realized Volatility
This notebook will pull summarized data from FinSpace's catalog and then use the analytic function realized_volatility to compute realized volatility for a group of tickers and exchange event types.

In [ ]:
#####----------------------------------------------------------
##### REPLACE WITH CORRECT IDS!
##### Dataset: "US Equity Time-Bar Summary - 1 min, 14 Symbols - Sample"
#####
#####----------------------------------------------------------
dataset_id = ''  
view_id    = ''

In [ ]:
# import needed libraries
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import pyspark.sql.functions as F
import pyspark.sql.types as T

# import time series libraries
from aws.finspace.timeseries.spark.analytics import *
from aws.finspace.timeseries.spark.windows import *

from aws.finspace.timeseries.spark.util import string_to_timestamp_micros

sumDF = finspace.read_view_as_spark(dataset_id = dataset_id, view_id = view_id)

# Filter and select
sDate = dt.datetime(2020, 1, 1)
eDate = dt.datetime(2020, 3, 1)

df = ( sumDF.filter( sumDF.date.between(sDate, eDate) ) )

# sample the data
df.show(5, truncate=False)

# Spark Analytics
All our analytic functions have help, lets look at the signatures for the functions we will use

![Workflow](./workflow.png)

In [ ]:
help(realized_volatility)

# Calculate Realized Volatility
Compute the realized volatility of the time series data.

In [ ]:
tenor = 15
numStd = 2

# analytics to calculate
realVolDef = realized_volatility( tenor, "end", "vwap" )

# group the sets of values
partitionList = ["ticker", "eventtype"]

tsDF = df

tsDF = compute_analytics_on_features(tsDF, "realized_volatility", realVolDef, partition_col_list = partitionList).cache()

tsDF.printSchema()

# Realized Volatility Graph
Plot realized volatility
1. Calculations are performed on the cluster
2. Results are then collected to the driver as a pandas DataFrame
3. Plot image created from pandas data
4. Plot image is sent to the 'local' notebook for display

This is all done for you.

In [ ]:
fTicker = 'AMZN'

# filter and bring data into a pandas dataframe for plotting
pltDF = ( tsDF
    .filter(tsDF.eventtype == "TRADE NB")
    .filter(tsDF.ticker == fTicker)
    .select( 'end', 'realized_volatility' )
).toPandas().set_index('end')

pltDF.index = pltDF.index.strftime("%Y-%m-%d %H:%m")

fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Realized Volatility
pltDF[[ 'realized_volatility' ]].plot(figsize=(12,6))

# labels and other items to make the plot readable
plt.title(f"{fTicker} Realized Vol (tenor: {tenor}, 1 min bars)")
plt.ylabel('Realized Vol')
plt.xlabel('Date/Time')
plt.xticks(rotation=30)
plt.subplots_adjust(bottom=0.2)

%matplot plt

# So why that spike?

[Amazon soars after huge earnings beat](https://www.cnbc.com/2020/01/30/amazon-amzn-q4-2019-earnings.html) (CNBC).  
- Amazon reported fourth-quarter results on Thursday that smashed analysts’ expectations.  
- The company’s profits rebounded during the quarter, while revenue climbed 21% year over year.  
- The outperforming results show Amazon’s big investments in one-day delivery are paying off.  

# Incorporate Events from Factset
Factset has the event data: [Factset Events](https://app.snowflake.com/marketplace/listing/GZT0Z28ANXM)

In [ ]:
# attribute name that contains the necessary catalog, schema and table data to get data from Snowflake.
snowflake_att_name   = 'Snowflake Table Attributes'

#####----------------------------------------------------------
##### REPLACE WITH CORRECT IDS!
#####----------------------------------------------------------

# dataset IDs of the data as registered in the FinSpace catalog
sym_ticker_region  = ''  
sym_coverage = ''
sym_entity = ''

ce_events = ''
ce_reports = ''
ce_sec_entity_hist = ''


## Utility functions
Use the Snowflake Metadata in Finspace to query for the data in snowflake, creating a Spark dataframe based on the snowflake table.


In [ ]:
def find_by_key_value(l, key, v):
    for n in l:
        if n[key] == v:
            return n
    
def get_field_by_name(f_list, title, name = 'name'):
    f = find_by_key_value(f_list, 'title', title)

    if f is not None:
        return f[name]

    return None

def get_field_values(f_list, field):
    for f in f_list:
        if f['field'] == field:
            return f['values']

def get_snowflake_query(dataset_id, att_name):
    sfAttrSet = finspace.attribute_set(att_name)
    
    if sfAttrSet is None:
        print(f'Did not find the attribute set with name: {att_name}')
        return None

    # get the dataset details
    dataset_details = finspace.client.describe_dataset_details(datasetId = dataset_id)

    if dataset_details is None:
        print(f'Did not find the dataset with id: {dataset_id}')
        return None
        
    # find the snowflake attributes related to the dataset
    attributes = dataset_details['datasetTypeContexts']
    
    sfAttr = find_by_key_value(attributes, 'id', sfAttrSet['id'])

    if sfAttr is None:
        print(f'Did not find the attribute set with name: {att_name} in the dataset with id: {dataset_id}')
        return None
    
    field_defs = sfAttr['definition']['fields']

    catalog = get_field_by_name(field_defs, 'Catalog')
    schema = get_field_by_name(field_defs, 'Schema')
    table = get_field_by_name(field_defs, 'Table')

    field_values = sfAttr['values']

    return f'"{get_field_values(field_values, catalog)[0]}"."{get_field_values(field_values, schema)[0]}"."{get_field_values(field_values, table)[0]}"'

def get_dataframe_from_snowflake(dataset_id, att_name='Snowflake Table Attributes'):
    # get the query for snowflake from the data in the attributeset
    sf_query = get_snowflake_query(dataset_id = dataset_id, att_name = att_name)

    # load the dataframe from snowflake using information from finspace
    df = ( spark.read.format(SNOWFLAKE_SOURCE_NAME) 
      .options(**sfOptions)
      .option('query',  sf_query)
      .load()
    )
    
    return df

## Create DataFrames from Snowflake Tables
These functions create Spark DataFrames from Snowflake tables, using the information about their location that was registered in FinSpace's catalog.

In [ ]:
# symbols
sym_ticker_region_df  = get_dataframe_from_snowflake(dataset_id = sym_ticker_region, att_name = snowflake_att_name)
sym_coverage_df = get_dataframe_from_snowflake(dataset_id = sym_coverage, att_name = snowflake_att_name)
sym_entity_df = get_dataframe_from_snowflake(dataset_id = sym_entity, att_name = snowflake_att_name)

# events
ce_sec_entity_hist_df = get_dataframe_from_snowflake(dataset_id = ce_sec_entity_hist, att_name = snowflake_att_name)
ce_reports_df = get_dataframe_from_snowflake(dataset_id = ce_reports, att_name = snowflake_att_name)
ce_events_df = get_dataframe_from_snowflake(dataset_id = ce_events, att_name = snowflake_att_name)

## Now the Factset data to the dataset with volatility 

Spark dataframe operations to join the data on necessary keys

In [ ]:
# first the symbol data....
symbol_df = ( sym_ticker_region_df
            .join(sym_coverage_df, ['FSYM_ID'] ) #    sym_ticker_region_df.FSYM_ID == sym_coverage_df.FSYM_ID)
            .withColumnRenamed('FSYM_ID', 'FSYM_ID_DELETE')
            .join(ce_sec_entity_hist_df, ce_sec_entity_hist_df.FSYM_ID ==  sym_coverage_df.FSYM_SECURITY_ID)
            .join(sym_entity_df, ['FACTSET_ENTITY_ID']) #sym_entity_df.FACTSET_ENTITY_ID == ce_sec_entity_hist_df.FACTSET_ENTITY_ID)
            ).drop('FSYM_ID_DELETE')

# Data about Amazon
ticker = 'AMZN'
region = 'US'

symbol_df.filter(symbol_df.TICKER_REGION == f'{ticker}-{region}' ).show(truncate=False)

In [ ]:
# Now pull in the events as well
events_df = ( symbol_df
            .join(ce_reports_df, ['FACTSET_ENTITY_ID']) #ce_reports_df.FACTSET_ENTITY_ID == ce_sec_entity_hist_df.FACTSET_ENTITY_ID)
            .join(ce_events_df, ['EVENT_ID']) #ce_events_df.EVENT_ID == ce_reports_df.EVENT_ID)
            )

In [ ]:
# what where the events over that same period?
print(f'{ticker}: {sDate} to {eDate}')

e_df = ( events_df
        .filter( events_df.TICKER_REGION == f'{ticker}-{region}' )
        .filter( events_df.EVENT_DATETIME_UTC.between(sDate, eDate) )
        .orderBy( events_df.ENTITY_PROPER_NAME, events_df.EVENT_DATETIME_UTC.asc() )
)

e_df.select(e_df.TICKER_REGION, e_df.FSYM_ID, e_df.FACTSET_ENTITY_ID, e_df.ENTITY_PROPER_NAME, e_df.EVENT_DATETIME_UTC, e_df.EVENT_TYPE, e_df.TITLE).show(truncate=False)

# Plot with Events
Adding the events as labeled veritcal lines to the original plot.

In [ ]:
fTicker = 'AMZN'
fRegion = 'US'
max_str_len = 30

# filter and bring data into a pandas dataframe for plotting
tradeDF = ( tsDF
    .filter(tsDF.eventtype == "TRADE NB")
    .filter(tsDF.ticker == fTicker)
    .select( 'end', 'realized_volatility' )
).toPandas().set_index('end')

evtDF = ( events_df
        .filter( events_df.TICKER_REGION == f'{fTicker}-{fRegion}' )
        .filter(events_df.EVENT_DATETIME_UTC.between(sDate, eDate) )
        .orderBy( events_df.ENTITY_PROPER_NAME, events_df.EVENT_DATETIME_UTC.asc() )
        .select( ['EVENT_DATETIME_UTC', 'TICKER_REGION', 'ENTITY_PROPER_NAME', 'EVENT_TYPE', 'TITLE'] )
).toPandas().set_index('EVENT_DATETIME_UTC')

# dataframe for plotting, concat the two
pltDF = pd.concat( [tradeDF, evtDF], axis=0).sort_index()

# index as string 
pltDF.index = pltDF.index.strftime("%Y-%m-%d %H:%m")

# for placing the labels on events
y_min = pltDF.realized_volatility.min()
y_max = pltDF.realized_volatility.max()
mid = (y_max - y_min) / 2
step = 0.1*mid

In [ ]:
# the plot
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Realized Volatility
pltDF[[ 'realized_volatility' ]].plot(figsize=(12,6), legend=None)

# labels and other items to make the plot readable
plt.title(f"{fTicker} Realized Vol (tenor: {tenor}, 1 min bars)")
plt.ylabel('Realized Vol')
plt.xlabel('Date/Time')
plt.xticks(rotation=30)
plt.subplots_adjust(bottom=0.2)

# get the locations of the events and plot vertical lines
row_indexes = pltDF.index.tolist()
events = pltDF.loc[pltDF.EVENT_TYPE.notnull()]

s = 2

for d in events.index:
    idx = row_indexes.index(d)
    plt.axvline(idx, linewidth=1, color='r', alpha=0.2)
    e = events.loc[d]
    t = f'({e.EVENT_TYPE}) {e.TITLE}'

    t = (t[:max_str_len] + '..') if len(t) > max_str_len else t
    y = y_max - (step * s)
    s = s+1
               
    plt.text(idx*1.01, y, t, rotation=0)
    
# 
%matplot plt

# Save the Data to Snowflake
Lets save the data we created for the plot into Snowflake

In [ ]:
# SOURCE DATAFRAME
# ---------------------------------------------
print('Source: Volatility DataFrame')
print(f'Rows: {tsDF.count():,}')
tsDF.show(5, False)

### WRITE to Snowflake

In [ ]:
# INTO: Snowflake Target
# ---------------------------------------------
sfDatabaseName = snowflake_database
sfTableName    = f'volatility_from_finspace'

print(f'Saving to Snowflake: {sfDatabaseName}..{sfTableName}')

( tsDF.write
    .format(SNOWFLAKE_SOURCE_NAME)
    .options(**sfOptions)
    .option("dbtable", f"{sfDatabaseName}..{sfTableName}")
    .mode('overwrite')
    .save()
)

### READ AGAIN from Snowflake

In [ ]:
# READ table from Snowflake again
# ---------------------------------------------
tsDF_SF = ( spark.read.format(SNOWFLAKE_SOURCE_NAME)
  .options(**sfOptions)
  .option("query",  f"{sfDatabaseName}..{sfTableName}")
  .load() )

print(f'Read from SNOWFLAKE: {sfDatabaseName}..{sfTableName}')
print(f'Rows: {tsDF_SF.count():,}')
tsDF_SF.show(5, False)

# Snapshot the Data
Take advantage of FinSpace's changsets and views to create a snapshot of snowflake table's data.


In [ ]:
#####----------------------------------------------------------
##### REPLACE WITH CORRECT IDS!
#####----------------------------------------------------------

# EMPLOYEES Dataset in FinSpace
dataset_id = ''

# get the data from snowflake
df = get_dataframe_from_snowflake(dataset_id = dataset_id, att_name = snowflake_att_name)

In [ ]:
# then add the changeset, to the dataset
changeset_id = finspace.ingest_dataframe(data_frame=df, dataset_id = dataset_id, change_type='REPLACE', wait_for_completion=True)

In [ ]:
# Create an auto-updating View if one does not exist
existing_snapshots = finspace.list_views(dataset_id = dataset_id, max_results=100)

autoupdate_snapshot_id = None

# does one exist?
for ss in existing_snapshots:
    if ss['autoUpdate'] == True:
        autoupdate_snapshot_id = ss['id']

# if no auto-updating view, create it
if autoupdate_snapshot_id is None:
    autoupdate_snapshot_id = finspace.create_auto_update_view(
        dataset_id = dataset_id,
        destination_type = "GLUE_TABLE",
        partition_columns = [],
        sort_columns = [],
        wait_for_completion = False)
    print( f"Created autoupdate_snapshot_id = {autoupdate_snapshot_id}" )
else:
    print( f"Exists: autoupdate_snapshot_id = {autoupdate_snapshot_id}" )


## Employees Dataset
The dataset in FinSpace

[EMPLOYEES Dataset](https://ak6gkzahz2s26bwtlnx5p6.us-east-1.amazonfinspace.com/dataset/jeyitu0/data)